In [1]:
import wntr
import wntr.network.controls as controls
import re

In [2]:
inp_file_path = "../Demand_patterns/ctown_map_with_controls.inp"
wn = wntr.network.WaterNetworkModel(inp_file_path)
cpa_file_path = "../Demand_patterns/ctown.cpa"

C:\ProgramData\Anaconda3\lib\site-packages\wntr\epanet\io.py:2085: UserWarning:

Not all curves were used in "../Demand_patterns/ctown_map_with_controls.inp"; added with type None, units conversion left to user



In [3]:
def get_plc_with_tag(a_tag):
    for plc in plcs:
        if a_tag in plc['Sensors']:
            return plc
        if a_tag in plc['Actuators']:
            return plc

In [4]:
def get_control_rule_with_tag(a_tag):
    for control in control_list:
        if a_tag in control['actuator_tag']:
            return control

In [5]:
def create_control_list():
    control_list = []
    for control in wn.controls():
        control_dict = {}
        control_dict['tank_tag'] = control[1].condition.name.split(":")[0]
        control_dict['operator'] = control[1].condition.name.split("level")[1][0:2]
        control_dict['value'] = control[1].condition.name.split("=")[1]
        control_dict['actuator_tag'] = str(control[1].actions()[0].target()[0])
        control_dict['actuator_value'] = str(control[1].actions()[0]).split("to")[1].strip()    
        control_list.append(control_dict)                     
    return control_list

In [6]:
def create_plc_list():
    section_exp = re.compile(r'\[(.*)\]')
    plc_exp = re.compile(r'^PLC')
    plcs = []
    nodes_section = False
    with open(cpa_file_path, 'r') as file_object:
            for line in file_object:
                if section_exp.match(line) and line == '[CYBERNODES]\n':
                    nodes_section=True
                    continue
                if section_exp.match(line) and line != '[CYBERNODES]\n':                
                    nodes_section=False
                if nodes_section and line[0] != ";":                
                    nodes=line.split(",")
                    if plc_exp.match(nodes[0].strip()):
                        plc_dict = {}
                        plc_dict['PLC'] = nodes[0].strip()
                        plc_dict['Sensors'] = nodes[1].strip().split(" ")
                        plc_dict['Actuators'] = nodes[2].strip().split(" ")[:-1]
                        plc_dict['Controls'] = []
                        plcs.append(plc_dict)                                                  
    return plcs

In [7]:
def set_control_list_in_plc(a_plc_dict, a_control_list):
    plc_controls = []
    for actuator in a_plc_dict['Actuators']:
        for control in a_control_list:
            control['actuator_tag']
            if actuator == control['actuator_tag']:
                plc_controls.append(control)                
    a_plc_dict['Controls'] = plc_controls 
    return a_plc_dict 

In [8]:
def set_dependencies_list_in_plc(a_plc_list, a_control_list):
    for plc in a_plc_list:
        dependencies_list = []        
        tag_set = set()        
        for control in plc['Controls']:
            tag_set.add(control['tank_tag'])
        if tag_set:            
            for tag in tag_set:
                for target_plc in a_plc_list:                    
                    for sensor in target_plc['Sensors']:                        
                        if tag == sensor:
                            dependency_dict = {}
                            dependency_dict['tag'] = tag
                            dependency_dict['PLC'] = target_plc['PLC']
                            dependencies_list.append(dependency_dict)        
        plc['Dependencies'] = dependencies_list                

In [9]:
control_list = create_control_list()
plc_list = create_plc_list()

for i in range(len(plc_list)):
    plc_list[i] = set_control_list_in_plc(plc_list[i], control_list)

set_dependencies_list_in_plc(plc_list, control_list)
    
for plc in plc_list:
    print(plc)
    print("\n")

{'PLC': 'PLC1', 'Sensors': [''], 'Actuators': ['PU1', 'PU2'], 'Controls': [{'tank_tag': 'T1', 'operator': '<=', 'value': '4.0', 'actuator_tag': 'PU1', 'actuator_value': 'Open'}, {'tank_tag': 'T1', 'operator': '>=', 'value': '6.3', 'actuator_tag': 'PU1', 'actuator_value': 'Closed'}, {'tank_tag': 'T1', 'operator': '<=', 'value': '1.0', 'actuator_tag': 'PU2', 'actuator_value': 'Open'}, {'tank_tag': 'T1', 'operator': '>=', 'value': '4.5', 'actuator_tag': 'PU2', 'actuator_value': 'Closed'}], 'Dependencies': [{'tag': 'T1', 'PLC': 'PLC2'}]}


{'PLC': 'PLC2', 'Sensors': ['T1'], 'Actuators': [], 'Controls': [], 'Dependencies': []}


{'PLC': 'PLC3', 'Sensors': ['T2'], 'Actuators': ['PU4', 'PU5', 'PU6', 'PU7'], 'Controls': [{'tank_tag': 'T3', 'operator': '<=', 'value': '3.0', 'actuator_tag': 'PU4', 'actuator_value': 'Open'}, {'tank_tag': 'T3', 'operator': '>=', 'value': '5.3', 'actuator_tag': 'PU4', 'actuator_value': 'Closed'}, {'tank_tag': 'T3', 'operator': '<=', 'value': '1.0', 'actuator_tag': 